<a href="https://colab.research.google.com/github/dragdowl1/s_devices/blob/main/Yulia_Draginda/Tesseract_SNILs_dict_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Распознавание фио в СНИЛс с помощью обученных tesseract и jamspell

In [ ]:
%%capture
!apt-get -qq install tesseract-ocr     # tesseract    
!apt-get -qq install imagemagick  

In [ ]:
%%capture
!pip install pytesseract
!pip install Pillow==9.0.0
# restart runtime

In [ ]:
%%capture
!pip install rapidfuzz
!pip install swig==3.0.6 # нужен именно swig3
!pip install jamspell
!pip install jiwer

In [ ]:
import re
import os
import csv
import PIL
import shutil
import difflib
import jamspell
import numpy as np
import pytesseract
import pandas as pd
from pathlib import Path
from jiwer import wer, cer
from zipfile import ZipFile
from pytesseract import Output
import matplotlib.pyplot as plt
from urllib.request import urlopen
from google.colab import files,drive
from IPython.display import clear_output
from PIL import Image, ImageEnhance, ImageOps

### Загрузка данных

In [ ]:
# Датасет СНИЛс
file_url = ' https://draginda.org/internship_materials/Clean_snils_data.zip'
file_path='/content/' 

zipresp = urlopen(file_url)    
tempzip = open("/tmp/tempfile.zip", "wb") 
tempzip.write(zipresp.read())
tempzip.close()
    
zf = ZipFile("/tmp/tempfile.zip")     
zf.extractall(path = file_path) 
zf.close()

In [ ]:
# провекрка уникальных символов
df_check = pd.read_csv('/content/clean_snils_labels.csv', sep=',')
letters = df_check[['fio1', 'fio2', 'fio3']].dropna()
b = pd.unique(letters.values.ravel()).tolist()
unique_chars = list(set().union(*b))
print(unique_chars)

['Я', 'Ы', 'З', 'Э', 'Т', 'Ш', 'Ц', 'Ё', 'Х', 'Г', 'П', 'У', 'Ф', 'Е', 'Д', 'Б', 'К', '-', 'Н', 'И', 'Ж', 'Ь', 'М', 'Ю', 'С', 'А', 'О', 'Ч', 'В', 'Й', 'Л', 'Р']


In [ ]:
# загрузка модели, обученной на словаре фио для tesseracta
!wget  https://draginda.org/internship_materials/fio_rus.traineddata
clear_output()

In [ ]:
# загрузка обученной модели фио для автокорректировщика jamspell
!wget  https://draginda.org/internship_materials/model_rus_fio.bin
clear_output()

In [ ]:
# загрузки отдельных моделей для имен, фамилий и отчеств для jamspell
!wget  https://draginda.org/internship_materials/model_rus_names.bin
!wget  https://draginda.org/internship_materials/model_rus_surnames.bin
!wget  https://draginda.org/internship_materials/model_rus_midnames.bin
clear_output()

## Подготовка файлов для tesseract и jamspell

In [ ]:
# файл с моделью копируем в папку tessdata
src = '/content/fio_rus.traineddata'
dst = '/usr/share/tesseract-ocr/4.00/tessdata/'
shutil.copy2(src, dst)

'/usr/share/tesseract-ocr/4.00/tessdata/fio_rus.traineddata'

In [ ]:
# создаем экземпляр класса корректироващика для фио и подгружаем обученную модель
jam_model_path = '/content/model_rus_fio.bin'
g_corrector = jamspell.TSpellCorrector()
g_corrector.LoadLangModel(jam_model_path)

True

## Подбор параметров

In [ ]:
!tesseract --help-psm 

Page segmentation modes:
  0    Orientation and script detection (OSD) only.
  1    Automatic page segmentation with OSD.
  2    Automatic page segmentation, but no OSD, or OCR. (not implemented)
  3    Fully automatic page segmentation, but no OSD. (Default)
  4    Assume a single column of text of variable sizes.
  5    Assume a single uniform block of vertically aligned text.
  6    Assume a single uniform block of text.
  7    Treat the image as a single text line.
  8    Treat the image as a single word.
  9    Treat the image as a single word in a circle.
 10    Treat the image as a single character.
 11    Sparse text. Find as much text as possible in no particular order.
 12    Sparse text with OSD.
 13    Raw line. Treat the image as a single text line,
       bypassing hacks that are Tesseract-specific.


In [ ]:
!tesseract --help-oem

OCR Engine modes: (see https://github.com/tesseract-ocr/tesseract/wiki#linux)
  0    Legacy engine only.
  1    Neural nets LSTM engine only.
  2    Legacy + LSTM engines.
  3    Default, based on what is available.


## Распознавание текстовых полей

In [ ]:
def calculate_accuracy(str1, str2):
    result = difflib.SequenceMatcher(None, str1, str2)
    return round(result.ratio(), 4)

In [ ]:
def recognize_text(images_path, labels_path, custom_config, WS, autocorr=False):

  counter = 0
  accs = []
  wers = []
  cers = []
  ws = WS+' '
  wl = set(ws)
  bad_full_dict = {}
  list_names= os.listdir(images_path)


  for ll in list_names:
    if 'fio' in ll:
      if counter>1:
        print("prev", acc, w, c)
      im_key = ll.split('.')[0]
      im_path = images_path+ll
      print(im_path)

      if os.path.isfile(im_path):
        lab_path = labels_path+im_key+".txt"
        with open(lab_path) as f:
          label = f.read()
          print(label)

        im = Image.open(im_path)
        # img = ImageOps.expand(im, border=4, fill='white') # не помогает
        # filter = ImageEnhance.Brightness(img) улучшение яркости
        # img = img.filter(1.2)
        # enhancer = ImageEnhance.Contrast(img)
        # contrast = 2  # коэффициент контрастности
        # img = enhancer.enhance(contrast)
        plt.imshow(im, cmap = 'gray')
        plt.axis('off')
        plt.show()

        ss = pytesseract.image_to_string(im, config = custom_config)
        result = ''.join([c for c in ss if c in wl])
        
        if autocorr:
            result = g_corrector.FixFragment(result)
        print(result)
        acc = calculate_accuracy(result, label)
        if result != "" and label !="":
          w = wer(result, label)
          c = cer(result,label)      
          wers.append(w)
          cers.append(c)
        print(acc, w, c)

        if acc<1:
          bad_dict = {}
          bad_dict['label']=label
          bad_dict['rec_res']=result
          bad_dict['acc'] =  acc       
          bad_full_dict[im_path] = bad_dict  
        clear_output()
        accs.append(acc) 
        counter+=1
        # if counter == 2: # debug
        #   break

  return bad_full_dict, accs, wers, cers 

In [ ]:
images_path = '/content/clean_snils_regions/'
labels_path = '/content/clean_snils_regions_labels/'

WS = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
custom_config = r'-c tessedit_char_whitelist=АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ --oem 1 -l fio_rus --psm 8  --dpi 500' # параметры распознавания tesseract для текстовый полей

bad_full_dict, accs, wers, cers  = recognize_text(images_path, labels_path, custom_config, WS, autocorr=False)

In [ ]:
print("Результаты без модели автозамены:\n")
print("Точность распознавания для текстовых полей СНИЛс:", np.round(sum(accs)/len((accs)),3))
print("Ошибка WER для текстовых полей СНИЛс:", np.round(sum(wers)/len((wers)),3))
print("Ошибка CER для текстовых полей СНИЛс:", np.round(sum(cers)/len((cers)),3))

Результаты без модели автозамены:

Точность распознавания для текстовых полей СНИЛс: 0.976
Ошибка WER для текстовых полей СНИЛс: 0.1
Ошибка CER для текстовых полей СНИЛс: 0.05


In [ ]:
# то, что не распозналось - пример
df = pd.DataFrame(bad_full_dict)
d1 = df.transpose()
d1.head(10)

,label,rec_res,acc
/content/clean_snils_regions/197_fio3.jpg,ВЕНИАМИНОВНА,ВЕНИДНИНОВНА,0.8333
/content/clean_snils_regions/283_fio2.jpg,ПАВЕЛ,,0.0
/content/clean_snils_regions/9_fio3.jpg,ЮРЬЕВИЧ,ЮРЪЕВИ,0.7692
/content/clean_snils_regions/80_fio2.jpg,АННА,ННЯ,0.5714
/content/clean_snils_regions/191_fio3.jpg,ИЗЗЕТОВИЧ,ИЗЗЕТОВИ,0.9412
/content/clean_snils_regions/245_fio3.jpg,ХАРИСОВНА,ХОРИСОВНЯ,0.7778
/content/clean_snils_regions/139_fio3.jpg,АЛЕКСЕЕВНА,АЛЕКСВЕРО,0.6316
/content/clean_snils_regions/108_fio1.jpg,ЗАЛИБЕКОВ,ЭЗАЛИБЕКОВ,0.9474
/content/clean_snils_regions/267_fio3.jpg,АЛЕКСЕЕВНА,АЛВКСВЕВНА,0.8
/content/clean_snils_regions/158_fio2.jpg,АНДРЕЙ,А,0.2857


### Распознавание с помощью обученного автокорректировщика

In [ ]:
bad_full_dict, accs, wers, cers  = recognize_text(images_path, labels_path, custom_config, WS, autocorr=True)

In [ ]:
print("Результаты при подключении обученной модели автозамены:\n")
print("Точность распознавания для текстовых полей СНИЛс:", np.round(sum(accs)/len((accs)),3))
print("Ошибка WER для текстовых полей СНИЛс:", np.round(sum(wers)/len((wers)),3))
print("Ошибка CER для текстовых полей СНИЛс:", np.round(sum(cers)/len((cers)),3))

Результаты при подключении обученной модели автозамены:

Точность распознавания для текстовых полей СНИЛс: 0.977
Ошибка WER для текстовых полей СНИЛс: 0.077
Ошибка CER для текстовых полей СНИЛс: 0.053


In [ ]:
# то, что не распозналось
df = pd.DataFrame(bad_full_dict)
d1 = df.transpose()
d1.head(10)

,label,rec_res,acc
/content/clean_snils_regions/229_fio1.jpg,ДОНГАА,ДОНГАК,0.8333
/content/clean_snils_regions/283_fio2.jpg,ПАВЕЛ,,0.0
/content/clean_snils_regions/9_fio3.jpg,ЮРЬЕВИЧ,ЮРЬЕВ,0.8333
/content/clean_snils_regions/80_fio2.jpg,АННА,АНЯ,0.5714
/content/clean_snils_regions/245_fio3.jpg,ХАРИСОВНА,БОРИСОВНА,0.7778
/content/clean_snils_regions/139_fio3.jpg,АЛЕКСЕЕВНА,АЛЕКПЕРОВ,0.6316
/content/clean_snils_regions/108_fio1.jpg,ЗАЛИБЕКОВ,АЛИБЕКОВ,0.9412
/content/clean_snils_regions/206_fio1.jpg,БРАЙЧУК,РАЙЧУК,0.9231
/content/clean_snils_regions/158_fio2.jpg,АНДРЕЙ,А,0.2857
/content/clean_snils_regions/231_fio1.jpg,КУНТЕЕВА,КУРТЕЕВА,0.875


### Распознавание с помощью отдельных моделей для имён, фамилий и отчеств

In [ ]:
# создаем отдельные экземпляры класса корректировщика для разных моделей
name_model_path = '/content/model_rus_names.bin'
g_name = jamspell.TSpellCorrector()
g_name.LoadLangModel(name_model_path)

surname_model_path = '/content/model_rus_surnames.bin'
g_surname = jamspell.TSpellCorrector()
g_surname.LoadLangModel(surname_model_path)

midname_model_path = '/content/model_rus_midnames.bin'
g_midname = jamspell.TSpellCorrector()
g_midname.LoadLangModel(midname_model_path)

True

In [ ]:
def recognize_text_by_fio(images_path, labels_path, custom_config, WS, autocorr=True):

  counter = 0
  accs = []
  wers = []
  cers = []
  ws = WS+' '
  wl = set(ws)
  bad_full_dict = {}
  list_names= os.listdir(images_path)


  for ll in list_names:
    if 'fio' in ll:
      if counter>1:
        print("prev", acc, w, c)
      im_key = ll.split('.')[0]
      im_path = images_path+ll
      print(im_path)
      # print('fio3'in im_key)

      if os.path.isfile(im_path):
        lab_path = labels_path+im_key+".txt"
        with open(lab_path) as f:
          label = f.read()
          print(label)

        im = Image.open(im_path)
        plt.imshow(im, cmap = 'gray')
        plt.axis('off')
        plt.show()

        ss = pytesseract.image_to_string(im, config = custom_config)
        result = ''.join([c for c in ss if c in wl])
        
        if 'fio1' in im_key:
            result = g_surname.FixFragment(result) # фамилия

        elif 'fio2' in im_key:
            result = g_name.FixFragment(result) # имя

        elif 'fio3' in im_key:
            result = g_midname.FixFragment(result) # отчество
        
        print(result)
        acc = calculate_accuracy(result, label)
        if result != "" and label !="":
          w = wer(result, label)
          c = cer(result,label)      
          wers.append(w)
          cers.append(c)
        print(acc, w, c)

        if acc<1:
          bad_dict = {}
          bad_dict['label']=label
          bad_dict['rec_res']=result
          bad_dict['acc'] =  acc       
          bad_full_dict[im_path] = bad_dict  
        clear_output()
        accs.append(acc) 
        counter+=1
        # if counter == 2: # debug
        #   break

  return bad_full_dict, accs, wers, cers 

In [ ]:
# распознавание с отдельными моделями
bad_full_dict, accs, wers, cers  = recognize_text_by_fio(images_path, labels_path, custom_config, WS)

In [ ]:
print("Результаты при подключении трёх отдельных моделей автозамен для имён, фамилий и отчеств:\n")
print("Точность распознавания для текстовых полей СНИЛс:", np.round(sum(accs)/len((accs)),3))
print("Ошибка WER для текстовых полей СНИЛс:", np.round(sum(wers)/len((wers)),3))
print("Ошибка CER для текстовых полей СНИЛс:", np.round(sum(cers)/len((cers)),3))

Результаты при подключении трёх отдельных моделей автозамен для имён, фамилий и отчеств:

Точность распознавания для текстовых полей СНИЛс: 0.975
Ошибка WER для текстовых полей СНИЛс: 0.085
Ошибка CER для текстовых полей СНИЛс: 0.052


In [ ]:
# то, что не распозналось
df = pd.DataFrame(bad_full_dict)
d1 = df.transpose()
d1.head(10)

,label,rec_res,acc
/content/clean_snils_regions/229_fio1.jpg,ДОНГАА,ДОНГАК,0.8333
/content/clean_snils_regions/283_fio2.jpg,ПАВЕЛ,,0.0
/content/clean_snils_regions/80_fio2.jpg,АННА,АНЯ,0.5714
/content/clean_snils_regions/245_fio3.jpg,ХАРИСОВНА,БОРИСОВНА,0.7778
/content/clean_snils_regions/8_fio1.jpg,АБАССИ,АБАСОВ,0.6667
/content/clean_snils_regions/139_fio3.jpg,АЛЕКСЕЕВНА,АЛЕКСВЕРО,0.6316
/content/clean_snils_regions/108_fio1.jpg,ЗАЛИБЕКОВ,АЛИБЕКОВ,0.9412
/content/clean_snils_regions/206_fio1.jpg,БРАЙЧУК,РАЙЧУК,0.9231
/content/clean_snils_regions/158_fio2.jpg,АНДРЕЙ,А,0.2857
/content/clean_snils_regions/231_fio1.jpg,КУНТЕЕВА,КУРТЕЕВА,0.875


## Результаты

In [ ]:
print("Сравнение моделей - точность распознавания слов:\n") # 1-WER
print("Тesseract со словарём:", str((1-0.1)*100)+'%\n')
print("Тesseract со словарём + обученный jamspell (одна модель для фио):", str(np.round((1-0.077)*100,3))+'%\n')
print("Тesseract со словарём + обученный jamspell (три разных модели):", str(np.round((1-0.085)*100,3))+'%\n')

Сравнение моделей - точность распознавания слов:

Тesseract со словарём: 90.0%

Тesseract со словарём + обученный jamspell (одна модель для фио): 92.3%

Тesseract со словарём + обученный jamspell (три разных модели): 91.5%

